In [1]:
import os
import base64
import requests
import pandas as pd
from io import StringIO
from google.colab import userdata

In [2]:
api_key = userdata.get('API-key')

In [3]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def explain_image_base64(question, base64_image):

    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    }

    payload = {
      "model": "gpt-4-vision-preview",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": question
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
              }
            }
          ]
        }
      ],
      "max_tokens": 1000
    }

    openai_url = "https://api.openai.com/v1/chat/completions"

    response = requests.post(openai_url, headers=headers, json=payload)
    return response.json()

def explain_image(question, image_path):
    base64_image = encode_image(image_path)
    return explain_image_base64(question, base64_image)

In [13]:
image_path = r"Image.jpg"

json_answer = explain_image("""
Create csv file from this image with 2 columns. The image contains the following information:
1. Latin word or words: this includes latin words or phrases, presented in different forms,
and sometimes accompanied by examples of usage of those words. These words and sentences
can span multiple lines. Some entries include grammatical information like genus (m,f,n),
type of a word (e.g. Adv for adverb), or the case it imposes (e.g. m.Abl. for mit Ablativ, m.Gen for mit Genetiv)
2. German translations. Corresponding translations for the Latin words, also potentially
spanning multiple lines. This includes translations for both the words and usage examples.
3. Examples from the other languages.
The target CSV file must have the following structure:
- No header for the columns
- Separator: use | separator
- Column 1 (Latin content):
   a. combine Latin words and their respective examples into a single column
   b. use the <br> tag to separate lines within the same entrz
   c. format usage examples in small green italic text using HTML tags
   d. grammatical information (genus, type, case) should be formatted as italic text. Use HTML tags.
- Column 2 (German translation):
   a. corresponding German translations, grouped similarily to Latin content
   b. Translations of the examples should be in small green italic text, matching the structure in Column 1
Ignore Examples from the other languages on the image.
Example of formatted entries:
memor, memoris <i>m.Gen</i><br><i><span style="color: rgb(170,170,170)">
Memor periculi sum.</span></i>|in Erinnerung an <br><span style="color: rgb(170,170,170)">
Ich erinnere mich an die Gefahr.</span>
""", image_path)
answer = json_answer['choices'][0]['message']['content']
print(answer)

Certainly! Here is how the entries would be formatted in the requested CSV format with the provided specifications:

```
admittere, admittō, admīsī, admīssum|hinzuziehen, zulassen<br><i><span style="color: rgb(170,170,170)">ad, mittere, e. admit</span></i>
interest <i>(m.Gen.)</i>|es ist wichtig (für jdn.)<br>meā interest|es ist wichtig für mich
nimis / nimium <i>Adv.</i>|<i>(all)zu, (all)zu sehr</i><br>nimium libertātis|zu viel Freiheit
interesse, intersum, interfui <i>(m.Dat.)</i>|dazwischen sein; teilnehmen (an)<br><i><span style="color: rgb(170,170,170)">Interesse, interessant</span></i>
iūdicium|das Gericht, das Urteil<br><i><span style="color: rgb(170,170,170)">iūs, iūstus, iūdex, iūdicāre</span></i>
pūnīre, pūniō|bestrafen<br><i><span style="color: rgb(170,170,170)">poena, e. punish, f./s. punīr</span></i>
tamen <i>Subj.</i>|obwohl, wenn auch
quīcumque, quaecumque, quodcumque|jeder, der; wer auch immer<br><i><span style="color: rgb(170,170,170)">quī, quae, quod</span></i>
licent

In [5]:
lines = answer.split('\n')
lines

['Latin Content|Gender|German Translations',
 'admīttere, admittō, admīsī, admissum||hinzu ziehen, zulassen',
 'interest (m. Gen.)|m|es ist wichtig (für jdn.)',
 'meā interest||es ist wichtig für mich',
 'nimis / nimium Adv.||(all)zu, (all)zu sehr',
 'nīmium lībertātis||zu viel Freiheit',
 'interesse, intersum, interfuī (m. Dat.)|m|dazwischen sein; teilnehmen (an)',
 'iūdicium||das Gericht, das Urteil',
 'pūnīre, pūniō||bestrafen',
 'tamenetsī Subj.||obwohl, wenn auch',
 'quīcumque, quaecumque, quodcumque||jeder, der; wer auch immer',
 'licentia||die Freiheit, die Willkür',
 'accendere, accendō, accendī, accēnsum||anfeuern, anzünden',
 'īrā accēnsus||zornentbrannt, zornerfüllt',
 'nimius, a, um||übermäßig, zu groß',
 'ārdēre, ārdeō, ārsī, ārsum||brennen',
 'quisquis, quidquid (quicquid) subst.||jeder, der; wer auch immer',
 'quippe Adv.||freilich, zumal',
 'parum Adv.||(zu) wenig',
 'quārē||weshalb, wodurch; relativer Satzanschluss: deshalb']

In [6]:
df = pd.read_csv(StringIO(answer), delimiter='|', names = ['L', 'G', 'D'], header = 0)

In [7]:
df

,L,G,D
0,"admīttere, admittō, admīsī, admissum",NaN,"hinzu ziehen, zulassen"
1,interest (m. Gen.),m,es ist wichtig (für jdn.)
2,meā interest,NaN,es ist wichtig für mich
3,nimis / nimium Adv.,NaN,"(all)zu, (all)zu sehr"
4,nīmium lībertātis,NaN,zu viel Freiheit
5,"interesse, intersum, interfuī (m. Dat.)",m,dazwischen sein; teilnehmen (an)
6,iūdicium,NaN,"das Gericht, das Urteil"
7,"pūnīre, pūniō",NaN,bestrafen
8,tamenetsī Subj.,NaN,"obwohl, wenn auch"
9,"quīcumque, quaecumque, quodcumque",NaN,"jeder, der; wer auch immer"


In [8]:
df['result'] = df.apply(lambda row:
    f'{row["L"]}<br><i><span style="color: rgb(170, 170, 0);">{row["G"]}</span></i>|{row["D"]}<br><span style="color: rgb(170, 170, 0);">',
    axis=1)



In [9]:
result_str = '\n'.join(df['result'])

In [10]:
print(result_str)

admīttere, admittō, admīsī, admissum<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">nan</span></i>|hinzu ziehen, zulassen<br><span style="color: rgb(170, 170, 0);">
interest (m. Gen.)<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">m</span></i>|es ist wichtig (für jdn.)<br><span style="color: rgb(170, 170, 0);">
meā interest<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">nan</span></i>|es ist wichtig für mich<br><span style="color: rgb(170, 170, 0);">
nimis / nimium Adv.<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">nan</span></i>|(all)zu, (all)zu sehr<br><span style="color: rgb(170, 170, 0);">
nīmium lībertātis<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">nan</span></i>|zu viel Freiheit<br><span style="color: rgb(170, 170, 0);">
interesse, intersum, interfuī (m. Dat.)<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">m</span></i>|dazwischen sein; teilnehmen (an)<br><span style="color: rgb(170, 170, 0);">
iūdicium<i>m.Gen</i><b